In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("Ewon4_Full_data", low_memory=False)
df = df.drop(df.columns[0], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340997 entries, 0 to 340996
Data columns (total 47 columns):
 #   Column                                               Non-Null Count   Dtype  
---  ------                                               --------------   -----  
 0   TimeInt                                              340997 non-null  int64  
 1   TimeStr                                              340997 non-null  object 
 2   Act_Receptnummer                                     340997 non-null  int64  
 3   Active_nummer_wisselblok                             340997 non-null  int64  
 4   Pauze                                                340997 non-null  int64  
 5   StangData.Correctie_Totale_lengte                    340997 non-null  float64
 6   StangData.Correctie_Totale_lengte_operator_trek1en3  340997 non-null  float64
 7   StangData.Offset_knipklem_knippositie_Operator       340997 non-null  float64
 8   StangData.Pos-UitloCor_2_trek24                      3

In [3]:
df.head()

,TimeInt,TimeStr,Act_Receptnummer,Active_nummer_wisselblok,Pauze,StangData.Correctie_Totale_lengte,StangData.Correctie_Totale_lengte_operator_trek1en3,StangData.Offset_knipklem_knippositie_Operator,StangData.Pos-UitloCor_2_trek24,StangData.Pos_inloCor_1_trek13,...,negeer_stangkwijt,onbekende_stang,stang_kwijt,stangbuf_2,stangbuff_0,stangbuff_1,stangkwijt_buf,stangkwijt_timer,totaal_aantal,source_file
0,1761290871,24/10/2025 07:27:51,74,10,0,-0.2,0.0,-1.0,-0.9,1.1,...,1,0,0,0,0,0,0,0,15512946,20251024 SVRM4-HisLog18.csv
1,1761290876,24/10/2025 07:27:56,74,10,0,-0.2,0.0,-1.0,-0.9,1.1,...,1,0,0,0,0,0,0,0,15512946,20251024 SVRM4-HisLog18.csv
2,1761290881,24/10/2025 07:28:01,74,10,0,-0.2,0.0,-1.0,-0.9,1.1,...,1,0,0,0,0,0,0,0,15512946,20251024 SVRM4-HisLog18.csv
3,1761290886,24/10/2025 07:28:06,74,10,0,-0.2,0.0,-1.0,-0.9,1.1,...,1,0,0,0,0,0,0,0,15512946,20251024 SVRM4-HisLog18.csv
4,1761290891,24/10/2025 07:28:11,74,10,0,-0.2,0.0,-1.0,-0.9,1.1,...,1,0,0,0,0,0,0,0,15512946,20251024 SVRM4-HisLog18.csv


In [4]:
df['knipteller']

0         1536
1         1536
2         1536
3         1536
4         1536
          ... 
340992    6024
340993    6024
340994    6024
340995    6024
340996    6024
Name: knipteller, Length: 340997, dtype: int64

In [5]:
df['goedgekeurd'].describe()

count    340997.000000
mean       8902.146550
std        8692.967901
min           0.000000
25%        2526.000000
50%        7681.000000
75%       13001.000000
max       52281.000000
Name: goedgekeurd, dtype: float64

In [6]:
df['totaal_aantal'].describe()

count    3.409970e+05
mean     1.556100e+07
std      5.081194e+04
min      1.551295e+07
25%      1.551456e+07
50%      1.553677e+07
75%      1.558302e+07
max      1.568746e+07
Name: totaal_aantal, dtype: float64

In [7]:
df_subset = df[['TimeInt', 'knipteller', 'goedgekeurd', 'totaal_aantal']]
df_subset

,TimeInt,knipteller,goedgekeurd,totaal_aantal
0,1761290871,1536,7681,15512946
1,1761290876,1536,7681,15512946
2,1761290881,1536,7681,15512946
3,1761290886,1536,7681,15512946
4,1761290891,1536,7681,15512946
...,...,...,...,...
340992,1762491576,6024,751,15687455
340993,1762491581,6024,751,15687455
340994,1762491586,6024,751,15687455
340995,1762491591,6024,751,15687455


In [8]:
# Bereken delta's met reset-logica
def safe_diff(series):
    d = series.diff()
    d[d < 0] = series[d < 0]  # reset detectie
    return d.fillna(0)         # eerste rij NaN -> 0

df["d_knip"] = safe_diff(df["knipteller"])
df["d_goed"] = safe_diff(df["goedgekeurd"])
df["d_total"] = df["totaal_aantal"].diff().clip(lower=0).fillna(0)

# Correcte simpele berekening van bad_spaak
df["bad_spaak"] = (df[["d_knip", "d_total"]].min(axis=1) - df["d_goed"]).clip(lower=0).astype(int)


In [9]:
# Filter alleen de rijen met defecte spaken
df_bad = df[df["bad_spaak"] > 0]

# Reset de index voor overzicht
df= df.reset_index(drop=True)

df

,TimeInt,TimeStr,Act_Receptnummer,Active_nummer_wisselblok,Pauze,StangData.Correctie_Totale_lengte,StangData.Correctie_Totale_lengte_operator_trek1en3,StangData.Offset_knipklem_knippositie_Operator,StangData.Pos-UitloCor_2_trek24,StangData.Pos_inloCor_1_trek13,...,stangbuff_0,stangbuff_1,stangkwijt_buf,stangkwijt_timer,totaal_aantal,source_file,d_knip,d_goed,d_total,bad_spaak
0,1761290871,24/10/2025 07:27:51,74,10,0,-0.2,0.0,-1.0,-0.9,1.1,...,0,0,0,0,15512946,20251024 SVRM4-HisLog18.csv,0.0,0.0,0.0,0
1,1761290876,24/10/2025 07:27:56,74,10,0,-0.2,0.0,-1.0,-0.9,1.1,...,0,0,0,0,15512946,20251024 SVRM4-HisLog18.csv,0.0,0.0,0.0,0
2,1761290881,24/10/2025 07:28:01,74,10,0,-0.2,0.0,-1.0,-0.9,1.1,...,0,0,0,0,15512946,20251024 SVRM4-HisLog18.csv,0.0,0.0,0.0,0
3,1761290886,24/10/2025 07:28:06,74,10,0,-0.2,0.0,-1.0,-0.9,1.1,...,0,0,0,0,15512946,20251024 SVRM4-HisLog18.csv,0.0,0.0,0.0,0
4,1761290891,24/10/2025 07:28:11,74,10,0,-0.2,0.0,-1.0,-0.9,1.1,...,0,0,0,0,15512946,20251024 SVRM4-HisLog18.csv,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340992,1762491576,07/11/2025 04:59:36,499,6,1,-0.4,0.7,-13.3,-4.3,-3.0,...,0,0,0,0,15687455,20251107 SVRM4-HisLog.csv,0.0,0.0,0.0,0
340993,1762491581,07/11/2025 04:59:41,499,6,1,-0.4,0.7,-13.3,-4.3,-3.0,...,0,0,0,0,15687455,20251107 SVRM4-HisLog.csv,0.0,0.0,0.0,0
340994,1762491586,07/11/2025 04:59:46,499,6,1,-0.4,0.7,-13.3,-4.3,-3.0,...,0,0,0,0,15687455,20251107 SVRM4-HisLog.csv,0.0,0.0,0.0,0
340995,1762491591,07/11/2025 04:59:51,499,6,1,-0.4,0.7,-13.3,-4.3,-3.0,...,0,0,0,0,15687455,20251107 SVRM4-HisLog.csv,0.0,0.0,0.0,0


In [11]:
df.to_csv('df_RIGHTTARGET.csv')